In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
%matplotlib inline

In [2]:
def swc_to_dataframe( file_name ) :
    col_names = ['id', 'type', 'x', 'y', 'z', 'radius', 'parent', 'seg_id', 'level', 'mode', 'timestamp', 'TFresindex']
    df = pd.read_csv( file_name, sep=r"\s+", comment = "#", names = col_names, index_col = 'id')
    return df

In [3]:
#model_directory = "/allen/programs/celltypes/workgroups/mousecelltypes/_UPENN_fMOST"
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST"
swc_directory = os.path.join( model_directory,  "SWC_files", "182724_HUSTregi_neurons_original")
image_directory = os.path.join( model_directory, "25um_V3" )

output_directory = os.path.join( model_directory, "SWC_files", "182724_HUSTregi_neurons_original_rendered")

In [14]:
#
# component types
#
types = {}
types['soma'] = {'type_id': 1, 'color': 'black'}
types['axon'] = {'type_id': 2, 'color': 'cyan'}
types['basal dendrite'] = {'type_id': 3, 'color': 'orangered'}
types['apical dendrite'] = {'type_id': 4, 'color': 'maroon'}

#
# image slices
#
slices = {}
slices['coronal'] = {'swc': 'z', 'image' : 'y' } #P
slices['horizontal'] = {'swc' : 'x', 'image' : 'z' } #I
slices['sagittal'] = {'swc': 'y', 'image' : 'x' } #L

#
# scale and flip to match image
#

transforms = {}
transforms['coronal'] = {'swc': 'z', 'scale': 25.0, 'shift': 0.0, 'transpose' : True, 'endpoint' : (1320-1)*10.0 } #A
transforms['horizontal'] = {'swc': 'x', 'scale': 25.0, 'shift': -500.0, 'transpose' : True, 'endpoint' : (800-1)*10.0 } #S
transforms['sagittal'] = {'swc': 'y', 'scale' : 25.0, 'shift' : 0.0, 'transpose' : True, 'endpoint' : (1140-1)*10.0 } #L


In [16]:
#
# Create morphology thumbnail for each SWC file in the swc directory
#

#
# read in image file
#
image_file = os.path.join(image_directory, "CCFTemplate25um.nii.gz" )
image = sitk.ReadImage( image_file )


for swc_file in glob.glob(os.path.join(swc_directory, "*.swc")):

    bb = os.path.basename( swc_file )
    
    # find the associated brain file
    bname = bb.split('_')[0]
        
    # set the output thumbnail png file name
    cname = bb.split('.')[0]
    rendered_file = os.path.join( output_directory, "%s.nii.gz" % cname )
    
    #if cname != '182724_3887-X5901-Y10339_reg' :
    #    continue
    
    print((cname,bname))
        
    #
    # read swc components into dataframe
    #
    df = swc_to_dataframe( swc_file )


    #
    # scale swc coordinates to match image volume
    #
    for sk, sv in transforms.items() :
        
        df[sv['swc']] = df[sv['swc']] + sv['shift']
 
        if sv['transpose'] :
            df[sv['swc']] = sv['endpoint'] - df[sv['swc']]
            
        df[sv['swc']] = df[sv['swc']] / sv['scale']
        
    
    #
    # create output image
    #
    rendered = sitk.Image( image.GetSize(), sitk.sitkUInt8, 0 )
    rendered.CopyInformation( image )
    
    #
    # fill in rendered image
    #
    size = image.GetSize()
    for index, row in df.iterrows() :
        
        pixel = (round(row['y']),round(row['z']),round(row['x']))
        
        isOutside = False
        for i in range(3) :
            if (pixel[i] >= size[i]) or (pixel[i] < 0) :
                isOutside = True
                break
        
        if not isOutside :
            rendered.SetPixel( pixel, int(row['type']))
    
    
    #
    # save output file
    #
    sitk.WriteImage( rendered, rendered_file, True )
    

('182724_1994-X7390-Y11953_reg', '182724')
('182724_2144-X6104-Y18293_reg', '182724')
('182724_2145-X6692-Y19094_reg', '182724')
('182724_2183-X6813-Y19983_reg', '182724')
('182724_2238-X7096-Y20268_reg', '182724')
('182724_2247-X6785-Y11803_reg', '182724')
('182724_2253-X7016-Y19293_reg', '182724')
('182724_2333-X7081-Y21185_reg', '182724')
('182724_2383-X9300-Y17890_reg', '182724')
('182724_2392-X5323-Y14549_reg', '182724')
('182724_2494-X6419-Y20983_reg', '182724')
('182724_2525-X8483-Y21088_reg', '182724')
('182724_2526-X7529-Y21792_reg', '182724')
('182724_2566-X9074-Y20204_reg', '182724')
('182724_2670-X5615-Y13225_reg', '182724')
('182724_2694-X9688-Y20468_reg', '182724')
('182724_2926-X5279-Y13761_reg', '182724')
('182724_2997-X5821-Y12613_reg', '182724')
('182724_3257-X9560-Y23057_reg', '182724')
('182724_3848-X8903-Y23965_reg', '182724')
('182724_3887-X5901-Y10339_reg', '182724')
('182724_4327-X4847-Y18972_reg', '182724')
('182724_4472-X5247-Y12162_reg', '182724')
('182724_45